# VGG16 Model

In [1]:
# Import library
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from IPython.display import Image

# Open file
import os
import PIL

# Model CNN (Deep learning network)
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dense,\
GlobalAveragePooling2D, Dropout, Flatten
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# set default random state 
from random import seed

In [2]:
# Create function plot loss function and accuracy score graph
def plot_graph(model_values):
    ''' 
    Input : Model_values of keras.callbacks.History
    Return : Graph of Loss function and accuracy score between training dataset and vaildation dataset
    '''
    # Subplots
    fig, ax = plt.subplots(1, 2, figsize=(14,5))
    
    # Plot loss
    plt.subplot(1, 2, 1)
    plt.plot(model_values.history['loss'], label='Training Loss');
    plt.plot(model_values.history['val_loss'], label='Testing Loss');
    plt.legend(fontsize=12, loc='upper right')
    plt.title('Training and Validation Loss')
    plt.xlabel('Epochs')
    plt.ylabel('Loss');
    
    # Plot MSE
    plt.subplot(1, 2, 2)
    
    plt.plot(model_values.history['accuracy'], label='Training Accuracy')
    plt.plot(model_values.history['val_accuracy'], label='Validation Accuracy')
    
    plt.legend(fontsize=12, loc='lower right')
    plt.title('Training and Validation Accuracy')
    plt.xlabel('Epochs')
    plt.ylabel('Accuracy');

## 00- First Check GPU

In [3]:
# https://www.tensorflow.org/guide/keras/sequential_model
# Due to we use Keras Sequential API, 
# We want to check GPU first before training our model for 
# impore efficiency and reduce time. 
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 16492743522068545721
xla_global_id: -1
]


In [19]:
import tensorflow as tf
print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))


Num GPUs Available:  0


## 01-Open Dataset

In [4]:
# Define the paths
original_dataset_path = r'C:\Users\HP\Desktop\Data\poultry_disease_detection_third_iteration\poultry_data'
train_path = r"C:\Users\HP\Desktop\Data\poultry_disease_detection_third_iteration\Train"
validation_path = r"C:\Users\HP\Desktop\Data\poultry_disease_detection_third_iteration\Validate"
test_path = r"C:\Users\HP\Desktop\Data\poultry_disease_detection_third_iteration\Test"

## 02- Preprocessing image dataset

In [5]:
# Set parameter of image 
# The default input size for this VGG16 model is 224x224
# https://keras.io/api/applications/vgg/
batch_size = 50 # Set the batch size for epoch cycle
img_height = 224 # Set the height of the picture
img_width = 224 # Set the width of the picture

# Rescale pixel to reduce image size before using in model
data_gen_train = ImageDataGenerator(rescale=1/255.)
data_gen_valid = ImageDataGenerator(rescale=1/255.)
data_gen_test = ImageDataGenerator(rescale=1/255.)

In [6]:
# Create training dataset
train_dataset = data_gen_train.flow_from_directory(train_path,
                                                   class_mode="categorical",
                                                   target_size=(img_height, img_width),
                                                   batch_size=batch_size)

# Create validation dataset
valid_dataset = data_gen_valid.flow_from_directory(validation_path,
                                                   class_mode="categorical",
                                                   target_size=(img_height, img_width),
                                                   batch_size=batch_size)

# Create testing dataset
test_dataset = data_gen_test.flow_from_directory(test_path,
                                                 class_mode="categorical",
                                                 target_size=(img_height, img_width),
                                                 batch_size=batch_size)


Found 6347 images belonging to 4 classes.


## 03- VGG Model Training

### Transfer learning

In [9]:
# import vgg16 model form keras API
# set input size of image of trianing is 224x224
# due to we want to use transfer learning process 
# we must add `include_top=False` because we wan to add our input data 
# we decide default weigh for model 
# from keras.applications import VGG16
vgg16_model = tf.keras.applications.VGG16(input_shape=(224,224,3),
                                          include_top=False, # Transfer learning
                                          weights="imagenet",
                                          )

In [10]:
# model summary
vgg16_model.summary()

Model: "vgg16"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0     

In [11]:
# fix weights and bias in hidden layers
# train specifically custom head and output layer
vgg16_model.trainable=False

### Add custom head and output layers

In [12]:
# Create output layer 
# We have 4 classes in our output we decide using activation="softmax" 
# for multi classification.
# Before output layer we decide use GlobalAveragePooling2D as 
# one type of flatten layer.
average_pooling_layer = tf.keras.layers.GlobalAveragePooling2D()(vgg16_model.output) # flatten
prediction_layer = tf.keras.layers.Dense(units=4, activation="softmax")(average_pooling_layer)

In [13]:
# Add Input layer and output layer 
model = tf.keras.models.Model(inputs=vgg16_model.input, 
                                    outputs=prediction_layer)

In [14]:
# compile the model         
model.compile(loss="categorical_crossentropy", 
              optimizer = tf.keras.optimizers.Adam(learning_rate=0.001),
              metrics=['accuracy'])

In [15]:
# check model summary
# total params: 14,716,740 (hidden layers)
# trainable params: 2,052 (input and output layers)
model.summary() 

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0     

In [16]:
# Save checkpoints during training
checkpoint_path = r'C:\Users\HP\Desktop\Data\poultry_disease_detection_third_iteration\models\vgg16\training_vgg16_cp\cp.ckpt'
checkpoint_dir = os.path.dirname(checkpoint_path)

# Create a callback that saves the model's weights
cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path,
                                                 save_weights_only=True,
                                                 verbose=1)

In [17]:
# Training model
# make sure you truely save checkpoint_path
history = model.fit(train_dataset,
          epochs=25,
          validation_data=valid_dataset,
          callbacks=[cp_callback])

Epoch 1/25


 11/127 [=>............................] - ETA: 1:05:39 - loss: 1.3809 - accuracy: 0.2927

KeyboardInterrupt: 

In [ ]:
# plot graph 
plot_graph(history)


In [ ]:
# save model
model.save(r"C:\Users\HP\Desktop\Data\poultry_disease_detection_third_iteration\models\vgg16\vgg_16.h5")

In [ ]:
# convert the history.history dict to a pandas DataFrame:     
hist_df = pd.DataFrame(history.history) 
hist_df.head()

In [ ]:
# # save history to csv: 
# hist_csv_file = 'history_vgg16.csv'
# with open(hist_csv_file, mode='w') as f:
#     hist_df.to_csv(f)

# Save history to CSV file
hist_csv_file = r"C:\Users\HP\Desktop\Data\poultry_disease_detection_third_iteration\models\vgg16\history_vgg16.csv"
hist_df.to_csv(hist_csv_file, index=False)  # Set index=False to exclude the index column

### Fine Tuning

In [ ]:
img_height = 224 # Set the height of the picture
img_width = 224 # Set the width of the picture

# load model
vgg16_model = tf.keras.applications.VGG16(input_shape=(img_height, img_width,3),
                                          include_top=False, # Transfer learning
                                          weights="imagenet",
                                          )  

# Let's take a look to see how many layers are in the base model
print("Number of layers in the base model: ", len(vgg16_model.layers))

# Fine-tune from this layer onwards
fine_tune_at = 15

# Freeze all the layers before the `fine_tune_at` layer
for layer in vgg16_model.layers[:fine_tune_at]:
    layer.trainable = False

# Make sure you have frozen the correct layers
for i, layer in enumerate(vgg16_model.layers):
    print(i, layer.name, layer.trainable)

# Add input layers and output layers
average_pooling_layer = tf.keras.layers.GlobalAveragePooling2D()(vgg16_model.output) # flatten
prediction_layer = tf.keras.layers.Dense(units=4, activation="softmax")(average_pooling_layer)
fineture_model = tf.keras.models.Model(inputs=vgg16_model.input, 
                                     outputs=prediction_layer)

# Compile the model         
fineture_model.compile(loss="categorical_crossentropy", 
                       optimizer = tf.keras.optimizers.Adam(learning_rate=0.001),
                       metrics=['accuracy'])

In [ ]:
# Save checkpoints during training
# follow value of vaildation scorce 
checkpoint = tf.keras.callbacks.ModelCheckpoint(r'C:\Users\HP\Desktop\Data\poultry_disease_detection_third_iteration\models\vgg16\ft_cp\vgg16_ft_checkpoint.h5', 
                             monitor= 'val_accuracy', 
                             mode= 'max', 
                             save_best_only = True, 
                             verbose= 1)

In [ ]:
# Training model
# make sure you truely save checkpoint_path
history_ft = fineture_model.fit(train_dataset,  
                             epochs=25, 
                             validation_data=valid_dataset, 
                             callbacks=[checkpoint])

In [ ]:
# plot graph 
plot_graph(history_ft)

# Overfitting between training and validation 
# Final accuracy after fine tuning 25 epochs is up score in training 1.00% (at 21 epoch)
# and vaildaion 0.94% (better than transfer learning)
# Loss function after training 25 epochs is down score in training 0.01% (at 21 epoch)
# and vaildaion 0.43% (better than transfer learning)
# Higher perfomance than baseline model  

In [ ]:
# save model after fine tuning 
fineture_model.save(r"C:\Users\HP\Desktop\Data\poultry_disease_detection_third_iteration\models\vgg16\vgg_16_ft.h5")

In [ ]:
# convert the history.history dict to a pandas DataFrame:     
hist_df = pd.DataFrame(history_ft.history) 

# # save to csv: 
# hist_csv_file = '../model/vgg16/history_vgg16_ft.csv'
# with open(hist_csv_file, mode='w') as f:
#     hist_df.to_csv(f)

# Save history to CSV file
hist_csv_file = r"C:\Users\HP\Desktop\Data\poultry_disease_detection_third_iteration\models\vgg16\history_vgg16_ft.csv"
hist_df.to_csv(hist_csv_file, index=False)  # Set index=False to exclude the index column

hist_df.head()